<a href="https://colab.research.google.com/github/kiarashgh98/Topic-modeling-/blob/main/5_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
!pip install bertopic
!pip install streamlit
!pip install --upgrade scipy plotly
!pip install --upgrade bertopic numba
!pip install --upgrade scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 22.8 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into ac

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from bertopic import BERTopic
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import plotly.graph_objects as go
import numpy as np
import math
import logging
from sentence_transformers import SentenceTransformer

pd.set_option('display.max_rows', None)

In [ ]:
# Suppress warnings from BERTopic
logging.getLogger('BERTopic').setLevel(logging.ERROR)

In [ ]:
# Initialize global variables
data_org, docs, my_model = None, None, None
messages = []

# Function to load and modify the model
def reduce_topics(b=None):
    # Enable other buttons
    wordcloud_button.disabled = False
    visualize_topics_button.disabled = False
    visualize_hierarchy_button.disabled = False
    clear_button.disabled = False
    save_button.disabled = False
    new_button.disabled = False


    with output:
        clear_output(wait=True)

    global data_org, docs, my_model, embeddings

    progress_bar.value = 0
    nc = int(nc_input.value) + 1
    outliers_ = '' if outliers_input.value == 'SciBERT' else 'outliers_'

    my_model = BERTopic.load(f"/content/drive/MyDrive/Kiarash/New Codes/Models and Outputs/mahsa_scibert_{outliers_}model")
    data_org = pd.read_csv(f"/content/drive/MyDrive/Kiarash/New Codes/mahsa_data_after_{outliers_}preprocessing.csv",low_memory = False)
    docs = data_org['cleaned_text']


    #topic_info = my_model.get_topic_info()
    #print(topic_info[['Topic', 'Count', 'Name']])

    update_status("Loading and reducing topics...", 40)


    my_model.reduce_topics(docs, nr_topics=nc)


    sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = sentence_model.encode(docs, show_progress_bar=True)



    update_status("Done!", 100)

# Function to update status messages
def update_status(message, progress):
    messages.append(message)
    progress_bar.value = progress
    with output:
        clear_output(wait=True)
        #for msg in messages:
            #print(msg)

def update_status1(message, progress):
    progress_bar.value = progress
    #with output:
        #print(message)




def visualize_topics_func(b):
    with output:
        update_status("Visualizing topics...", 50)

        # Assuming my_model.visualize_topics() returns a Plotly figure
        fig = my_model.visualize_topics()

        update_status("Intertopic distance map generation completed.", 100)
        display(fig)

def visualize_hierarchy_func(b):
    with output:
        update_status("Visualizing hierarchy...", 50)

        hierarchical_topics = my_model.hierarchical_topics(docs)
        fig = my_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

        update_status("Hierarchy visualization completed.", 100)
        display(fig)

def new_button_func(b):

  with output:
        update_status("Visualizing new plot...", 50)


        fig = my_model.visualize_documents(docs, embeddings=embeddings)


        update_status("New button visualization completed.", 100)
        display(fig)






# Function to visualize word clouds
def visualize_wc(b):
    global messages, cluster_numbers, frequent_words_list

    messages = []
    cluster_numbers = []
    frequent_words_list = []


    update_status("Generating WordClouds...", 60)

    topics = my_model.get_topics()
    num_topics = len(topics) - 1

    if num_topics % 2 == 0:
        num_rows = num_topics // 2
    else:
        num_rows = (num_topics // 2 ) + 1


    num_cols = 2 if num_topics > 1 else 1

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, num_rows * 3))
    axes = axes.flatten()



    for i in range(num_topics):
        try:
            if i < num_topics and topics[i]:
                text = {word: value for word, value in topics[i]}
                wordcloud = WordCloud(width=800, height=400, background_color="white", max_words=1000).generate_from_frequencies(text)
                axes[i].imshow(wordcloud, interpolation='bilinear')
                axes[i].axis('off')
                axes[i].set_title(f'Cluster {i+1}')


                 # Save frequent words along with their frequency
                frequent_words = ', '.join([f'{word} ({value})' for word, value in text.items()])
                cluster_numbers.append(f'Cluster {i+1}')
                frequent_words_list.append(frequent_words)



            else:
                raise ValueError("Empty topic retrieved")
        except Exception as e:
            axes[i].text(0.5, 0.5, str(e), horizontalalignment='center', verticalalignment='center')
            axes[i].axis('off')

    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    with output:
        plt.show()

    update_status1("Word Cloud Generation completed.", 100)

# Function to clear the output
def clear_figures(b):
    with output:
        clear_output()
        #print("Figures cleared.")

# Function to save the model and data
def save_model(b):
    clear_figures(b)

    with output:
        clear_output(wait=True)

    nc = int(nc_input.value) + 1
    outliers_ = '' if outliers_input.value == 'SciBERT' else 'outliers_'

    df_to_save = data_org.copy()

    print("Saving model, frequent words and probabilities...")
    my_model.save(f"/content/drive/MyDrive/Kiarash/New Codes/Models and Outputs/Saved Models/scibert_{nc}_{outliers_}model")
    df_to_save['probs'] = my_model.probabilities_.max(axis=1)
    df_to_save['topics'] = my_model.topics_
    df_to_save.to_csv(f"/content/drive/MyDrive/Kiarash/New Codes/Models and Outputs/Saved Models/scibert_{nc}_{outliers_}probs.csv")


    df_frequent_words = pd.DataFrame({
        'Cluster': cluster_numbers,
        'Frequent Words': frequent_words_list
    })

    df_frequent_words.to_csv(f'/content/drive/MyDrive/Kiarash/New Codes/Models and Outputs/Saved Models/scibert_{nc}_{outliers_}freqwords.csv', index=False)


    print("Saved successfully!")


In [ ]:
# Create widgets for inputs
nc_input = widgets.IntText(value=8, description='Number of Clusters:')
outliers_input = widgets.Dropdown(options=['SciBERT', 'SciBERT_Outliers'], value='SciBERT', description='Models:')
progress_bar = widgets.FloatProgress(value=0, min=0, max=100, description='Progress:')
output = widgets.Output()

# Create buttons for actions with custom styling
button_styles = {
    'description_width': 'initial'  # allows long descriptions to not overlap
}

reduce_topics_button = widgets.Button(description="Reduce Topics", button_style='info', style=button_styles)
wordcloud_button = widgets.Button(description="Word Cloud", button_style='info', style=button_styles, disabled=True)
new_button = widgets.Button(description="Visualize documents", button_style='info', style=button_styles, disabled=True)
visualize_topics_button = widgets.Button(description="Intertopic Distance Map", button_style='info', style=button_styles, disabled=True)
visualize_hierarchy_button = widgets.Button(description="Hierarchical Clustering", button_style='info', style=button_styles, disabled=True)
clear_button = widgets.Button(description="Clear Figures", button_style='warning', style=button_styles, disabled=True)
save_button = widgets.Button(description="Save Model", button_style='success', style=button_styles, disabled=True)


reduce_topics_button.on_click(reduce_topics)
wordcloud_button.on_click(visualize_wc)
new_button.on_click(new_button_func)
visualize_topics_button.on_click(visualize_topics_func)
visualize_hierarchy_button.on_click(visualize_hierarchy_func)
clear_button.on_click(clear_figures)
save_button.on_click(save_model)

# Here you can do a comparison! 🙂

In [ ]:
# Organize layout using VBox and HBox
button_box = widgets.HBox([reduce_topics_button, wordcloud_button, visualize_topics_button, new_button, visualize_hierarchy_button, clear_button, save_button])
input_box = widgets.VBox([nc_input, outliers_input, progress_bar])
display(widgets.VBox([input_box, button_box, output]))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/529 [00:00<?, ?it/s]

Batches:   0%|          | 0/529 [00:00<?, ?it/s]